<a href="https://colab.research.google.com/github/YoonTae-Hwang/trading_repository/blob/master/RSI_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Strategy : RSI

1. RSI Introduction 
 - RSI(Relative Strength index: 상대강도지수) 
 - 상대적인 주가 상승/하락의 강도를 수치화한 지표
 - 시장 추세의 상대적인 강도를 직관적으로 보여주는 기술적 지표

2. RSI Method
 - Define
    * U : N일간 가격대비 상승폭
    * L : N일간 가격대비 하락폭
    * AU , AD : U와 D의 평균값.
    * RS(Realative strength) = $\frac{AU}{AD}$
  - RSI
    *  $\frac{RS}{(1 + RS)}*100$ 
    *  $\frac{AU}{(AU + AD)}*100$
  - Characteristics
     * RSI <= 30 : 과매도 상태(Over Sold)   -> 매수포지션
     * RSI >= 70 : 과매수 상태(Over bought) -> 매도포지션
     * RSI 지표는 에서는 기간을 임의로 설정할 수 있다. RSI지표의 개발자인 월레스 와일더는 14일을 사용할 것을 권장하였다. 

3. Type of index - typical classification
 1. Leading index : 가격 변동에 선행하는 지표로 미래를 예측하기 위해 사용되며, 이러한 지표들은 보합이나 변동이 거의 없는 상태에서 유용함. 이를 통해서 주식의 가격이 급격히 상승하거나 급격히 하락하는 지점들을 확인할 수 있음.
 2. Lagging index : 가격의 상승 혹은 하락 추세가 지속되는지 약화되는지 확인할 때 사용할 수 있으므로 특정 추세기간에서 유용하게 사용할 수 있다.

4. Type of index - How are the index created?
 1. Oscilators Index : 기술적 지표의 일종으로 과매수와 과매도를 수치적으로 표시한다. 일반적으로 0에서 100까지 값을 갖도록 한다.
 2. Non-Bounded Index: 추세의 강함과 약한정도를 표현해주는 지표로 매수 시점과 매도 시점의 신호를 나타내는대 도움을 준다. 

5. Application of RSI
 1. Failure Swing Top: RSI >= 70에서 고점을 형성한 뒤 하락하는 경우에 직전 고점 형성부분의 최저점을 하향 돌파하는 지점에서 매매를 한다.[이미지첨부]

 2. Failure Swing Bottom: RSI <= 30에서 저점을 형성한 뒤 상승하는 경우에 직전 저점 형성부분의 최고점을 상향 돌파하는 시점에서 매매를 한다. [이미지첨부]

 3. divergence : divergence는 가격이 상승해 최고가격을 나타내고 있지만 RSI 지표는 상승하지 못하고 있는 상태이다. 이런 경우에 추세가 약화됨을 의미한다. Oscilators index들에서는 항상 divergence라는 개념이 존재한다. 일반적으로 divergence는 **주식 가격의 Bounded를 벗어나는 경우를 찾는데 매우 유용**하며, 중요한 점은 divergence가 확인한 후에 매매를 해야 한다는 점이다. 이런 의미로 **Failure Swing 전략**을 동반한다. 

 6. Cautions
    - RSI는 가격이 보합, 소폭의 등락만을 하는 상태에서는 RSI값이 50% 중심으로 움직이기 때문에 이러한 경우에 사용하기 곤란하다. 
    - RSI가 70보다 높다는 것은 차익 실현 매물이 나올것을 가정하는 것이다. 하지만 실제 상황에서는 다양한 이유(호재 등)로 RSI가 계속해서 상승할 수 있기 때문에 이 기술적 지표를 독립적으로 사용하기 보다는 보조적인 판단근거로 사용하는 것을 권장하기도 한다. 
    - 예를들어, 주식이 오른다는 확신이 있는 경우에 실제로 RSI가 70을 돌파하는 경우 단기 하락을 생각해 분할 매도를 한다. 이는 Failuere Swing과 같이 사용하면 된다.

### Before we start

1. Entry :
   - RSI <= 30  
2. Exit : 
  - RSI >= 70
3. Position Sizing :
  - buy / sell Apple 100 shares at time 

4. commission :
  - 0.01
5. Indicator setting :
  - period = 21

In [14]:
!pip install backtrader
#python library
import datetime
import os.path
import sys
import backtrader as bt
import matplotlib
import matplotlib.pyplot as plt

In [0]:
#RSI Strategy 
class rsi(bt.Strategy):
    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period = 14)

    def next(self):
        if not self.position:
            if self.rsi <= 30:
                self.buy(size = 100)
        else:
            if self.rsi >= 70:
                self.sell(size = 100)


In [28]:
if __name__ == "__main__":
    cerebro = bt.Cerebro()

    cerebro.addstrategy(rsi)

    data = bt.feeds.Quandl(
        dataname = "AAPL",
        fromdate = datetime.datetime(2000,1,1),
        todate = datetime.datetime(2020,4,10),
        buffered = True
    )

    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.01)
    cerebro.run()

    portvalue = cerebro.broker.getvalue()
    pnl = portvalue - 100000.0

    print('Final Portfolio Value: ${}'.format(portvalue))
    print('P/L: ${}'.format(pnl))    

    try:
      cerebro.plot(style='candlestick')
    except:
      pass


Final Portfolio Value: $104191.74630971368
P/L: $4191.746309713679


1. 마지막 줄의 다음과 같은 코드의 경우에 쥬피터노트북이나 파이참에서 실행하면 잘 싱행됨을 알 수 있다. 구글 코랩에서는 여러가지 방법을 사용해봐도 출력이 되지 않았다.

```
cerebro.plot(style='candlestick')
```

2. 최종적으로 벡테스팅 결과 \$4191.746309713679 만큼의 돈을 벌 었지만 실제로 나의 long / short 포지션의 결과로 인해 다른사람에게 미치는 영향을 고려하지 않았다는 점, closed를 기준으로 하였기 때문에 현실시장에서 원하는 시점에 즉각 구입하기가 힘들 수 있다는점, 실제 애플의 주가는 2000년대에 \$1이고 현재 대략 \$260라는 점을 고려할 때 시장을 이기지는 못했다.